In [1]:
setwd('..')
getwd()
library(ggplot2)

[1] "/Volumes/Macintosh_HDD/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/7_microbiomes_w_coral_size_lat"

In [2]:
library(ggplot2)
library(svglite)
library(gridExtra)
library(grid)

In [3]:
#Define function to ensure plots are a consistent size (useful for overlay, multipanels, etc)
set_panel_size <- function(p=NULL, g=ggplotGrob(p), file=NULL, 
                           margin = unit(1,"mm"),
                           width=unit(4, "cm"), 
                           height=unit(4, "cm")){

  panels <- grep("panel", g$layout$name)
  panel_index_w<- unique(g$layout$l[panels])
  panel_index_h<- unique(g$layout$t[panels])
  nw <- length(panel_index_w)
  nh <- length(panel_index_h)

if(getRversion() < "3.3.0"){

   # the following conversion is necessary
   # because there is no `[<-`.unit method
   # so promoting to unit.list allows standard list indexing
   g$widths <- grid:::unit.list(g$widths)
   g$heights <- grid:::unit.list(g$heights)

   g$widths[panel_index_w] <-  rep(list(width),  nw)
   g$heights[panel_index_h] <- rep(list(height), nh)

} else {

   g$widths[panel_index_w] <-  rep(width,  nw)
   g$heights[panel_index_h] <- rep(height, nh)

}

  if(!is.null(file))
    ggsave(file, g, 
           width = convertWidth(sum(g$widths) + margin, 
                                unitTo = "in", valueOnly = TRUE),
           height = convertHeight(sum(g$heights) + margin,  
                                  unitTo = "in", valueOnly = TRUE))

  g
}

In [4]:
#Import the data
read.csv('output/microbiome_age_v2.csv', header=T)->data
head(data)

,X.SampleID,PD_whole_tree,observed_otus,Chao1,SampleID,BarcodeSequence,runID,sequence_center_ID,sequence_center_ID_base,collection_id,ellip.h,collected_by,relevant_collaborators,visibility,visibility_method,env_biome,env_feature,prop_Colony_maximum_GCMP_recorded,prop_Colony_maximum_diameter_universal,prop_Colony_maximum_diameter,Description
1,E1.6.Iso.pali.1.20140803.M.2,37.90184,222,558.6,E1.6.Iso.pali.1.20140803.M.2,TACACAGTAG,ALVCP,Pollock117,Pollock117,Unknown,⋯,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
2,E1.6.Iso.pali.1.20140803.S.2,43.177,275,541.1429,E1.6.Iso.pali.1.20140803.S.2,CAGTCTACAT,ALVCP,Pollock119,Pollock119,Unknown,⋯,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
3,E1.6.Iso.pali.1.20140803.T.2,11.77071,72,237,E1.6.Iso.pali.1.20140803.T.2,TACTGCAGCG,ALVCP,Pollock118,Pollock118,Unknown,⋯,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
4,E10.Sed.1.20150627,74.79289,614,2318.721,E10.Sed.1.20150627,GATGTAGCGT,AN5UL,Pollock377_LH_268_SED,Pollock377,North_Bay_20150627,⋯,Joe_Pollock,Unknown,20,Diver_Estimate,coral_reef,animal-associated_habitat,Unknown,Unknown,Unknown,None
5,E10.Sed.1.20150628,86.31142,647,2820.015,E10.Sed.1.20150628,GTATCGTCGT,ALVCP,Pollock8,Pollock8,North_Bay_20150628,⋯,Joe_Pollock,Unknown,20,Diver_Estimate,coral_reef,animal-associated_habitat,Unknown,Unknown,Unknown,None
6,E10.Sed.1.20150630,77.96732,633,1653.509,E10.Sed.1.20150630,TCTCTGTGCA,AN5UL,Pollock419_LH_305_SED,Pollock419,Far_Flats_20150630,⋯,Joe_Pollock,Unknown,15,Diver_Estimate,coral_reef,animal-associated_habitat,Unknown,Unknown,Unknown,None


In [5]:
#remove Unknown size
subset(data,colony_width_max != "Unknown")->data_no_unknown_size

#remove Unknown functional group
subset(data_no_unknown_size,functional_group_sensu_darling != "Unknown")->data_no_unknown_size_or_functional_group

In [6]:
# Subset to only include Scleractinians
subset(data, taxonomy_string_to_order == "Cnidaria_Anthozoa_Scleractinia")->data_scleractinia
subset(data_scleractinia,  BiologicalMatter != "Whole_Coral")->data_scleractinia_compartments

In [7]:
nrow(data_scleractinia_compartments)

[1] 614

# Latitude

In [9]:
# PD_whole_tree vs. latitude
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Tissue")->data_corals_only_tissue
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Mucus")->data_corals_only_mucus
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Skeleton")->data_corals_only_skeleton

print("Tissue")
nrow(data_corals_only_tissue)
cor.test(as.numeric(as.character(data_corals_only_tissue$latitude)), as.numeric(as.character(data_corals_only_tissue$PD_whole_tree)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)
print("Mucus")
nrow(data_corals_only_mucus)
cor.test(as.numeric(as.character(data_corals_only_mucus$latitude)), as.numeric(as.character(data_corals_only_mucus$PD_whole_tree)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)
print("Skeleton")
nrow(data_corals_only_skeleton)
cor.test(as.numeric(as.character(data_corals_only_skeleton$latitude)), as.numeric(as.character(data_corals_only_skeleton$PD_whole_tree)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)

[1] "Tissue"


[1] 199


	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_tissue$latitude)) and as.numeric(as.character(data_corals_only_tissue$PD_whole_tree))
t = 2.4105, df = 197, p-value = 0.01685
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.03089782 0.30125792
sample estimates:
      cor 
0.1692603 


[1] "Mucus"


[1] 207


	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_mucus$latitude)) and as.numeric(as.character(data_corals_only_mucus$PD_whole_tree))
t = 3.9839, df = 205, p-value = 9.416e-05
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.1366930 0.3901728
sample estimates:
     cor 
0.268066 


[1] "Skeleton"


[1] 208


	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_skeleton$latitude)) and as.numeric(as.character(data_corals_only_skeleton$PD_whole_tree))
t = 2.7801, df = 206, p-value = 0.005938
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.05555803 0.31797700
sample estimates:
      cor 
0.1901619 


In [18]:
plot1 <-ggplot(data_scleractinia_compartments, aes(x=as.numeric(as.character(latitude)), y=as.numeric(as.character(PD_whole_tree)), color=BiologicalMatter)) +
    geom_point(shape=1) +
    geom_smooth(method=lm,   # Add linear regression lines
                se=TRUE)    # Add shaded confidence region

g1 <- set_panel_size(plot1)
ggsave(g1, file="/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/7_microbiomes_w_coral_size_lat/output/figures/raw_output/NEW_latitude_v_pd_color.svg")

Saving 7 x 7 in image


In [10]:
# observed_otus vs. latitude
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Tissue")->data_corals_only_tissue
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Mucus")->data_corals_only_mucus
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Skeleton")->data_corals_only_skeleton

print("Tissue")
nrow(data_corals_only_tissue)
cor.test(as.numeric(as.character(data_corals_only_tissue$latitude)), as.numeric(as.character(data_corals_only_tissue$observed_otus)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)
print("Mucus")
nrow(data_corals_only_mucus)
cor.test(as.numeric(as.character(data_corals_only_mucus$latitude)), as.numeric(as.character(data_corals_only_mucus$observed_otus)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)
print("Skeleton")
nrow(data_corals_only_skeleton)
cor.test(as.numeric(as.character(data_corals_only_skeleton$latitude)), as.numeric(as.character(data_corals_only_skeleton$observed_otus)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)

[1] "Tissue"


[1] 199


	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_tissue$latitude)) and as.numeric(as.character(data_corals_only_tissue$observed_otus))
t = 2.5752, df = 197, p-value = 0.01075
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.04244037 0.31173023
sample estimates:
     cor 
0.180465 


[1] "Mucus"


[1] 207


	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_mucus$latitude)) and as.numeric(as.character(data_corals_only_mucus$observed_otus))
t = 3.691, df = 205, p-value = 0.0002863
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.1172480 0.3732894
sample estimates:
      cor 
0.2496267 


[1] "Skeleton"


[1] 208


	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_skeleton$latitude)) and as.numeric(as.character(data_corals_only_skeleton$observed_otus))
t = 3.0264, df = 206, p-value = 0.00279
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.07231161 0.33301694
sample estimates:
      cor 
0.2063231 


In [23]:
plot1 <-ggplot(data_scleractinia_compartments, aes(x=as.numeric(as.character(latitude)), y=as.numeric(as.character(observed_otus)), color=BiologicalMatter)) +
    geom_point(shape=1) +
    geom_smooth(method=lm,   # Add linear regression lines
                se=TRUE)    # Add shaded confidence region

g1 <- set_panel_size(plot1)
ggsave(g1, file="/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/7_microbiomes_w_coral_size_lat/output/figures/raw_output/NEW_latitude_v_observed_otus_color.svg")

Saving 7 x 7 in image


# Size

In [20]:
# All together: size vs. pd_whole_tree
print("All_compartments")
cor.test(as.numeric(as.character(data_scleractinia_compartments$prop_Colony_maximum_diameter_universal)), as.numeric(as.character(data_scleractinia_compartments$PD_whole_tree)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)


[1] "All_compartments"



	Pearson's product-moment correlation

data:  as.numeric(as.character(data_scleractinia_compartments$prop_Colony_maximum_diameter_universal)) and as.numeric(as.character(data_scleractinia_compartments$PD_whole_tree))
t = -2.7722, df = 612, p-value = 0.005737
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.18882560 -0.03252428
sample estimates:
       cor 
-0.1113636 


In [16]:
# By compartment: size vs. pd_whole_tree
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Tissue")->data_corals_only_tissue
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Mucus")->data_corals_only_mucus
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Skeleton")->data_corals_only_skeleton

print("Tissue")
cor.test(as.numeric(as.character(data_corals_only_tissue$prop_Colony_maximum_diameter_universal)), as.numeric(as.character(data_corals_only_tissue$PD_whole_tree)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)
print("Mucus")
cor.test(as.numeric(as.character(data_corals_only_mucus$prop_Colony_maximum_diameter_universal)), as.numeric(as.character(data_corals_only_mucus$PD_whole_tree)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)
print("Skeleton")
cor.test(as.numeric(as.character(data_corals_only_skeleton$prop_Colony_maximum_diameter_universal)), as.numeric(as.character(data_corals_only_skeleton$PD_whole_tree)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)

[1] "Tissue"



	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_tissue$prop_Colony_maximum_diameter_universal)) and as.numeric(as.character(data_corals_only_tissue$PD_whole_tree))
t = -2.7751, df = 197, p-value = 0.006049
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.32430538 -0.05639631
sample estimates:
       cor 
-0.1939647 


[1] "Mucus"



	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_mucus$prop_Colony_maximum_diameter_universal)) and as.numeric(as.character(data_corals_only_mucus$PD_whole_tree))
t = 0.069759, df = 205, p-value = 0.9445
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.1315853  0.1411484
sample estimates:
        cor 
0.004872139 


[1] "Skeleton"



	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_skeleton$prop_Colony_maximum_diameter_universal)) and as.numeric(as.character(data_corals_only_skeleton$PD_whole_tree))
t = -1.7414, df = 206, p-value = 0.08311
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.25235037  0.01585686
sample estimates:
       cor 
-0.1204441 


In [19]:
# size vs. pd_whole_tree
#Clear plot
plot1<-0

#Plot
plot1 <-ggplot(data_scleractinia_compartments, aes(x=as.numeric(as.character(prop_Colony_maximum_diameter_universal)), y=as.numeric(as.character(PD_whole_tree)), color=BiologicalMatter)) +
    geom_point(shape=1) +
    geom_smooth(method=lm,   # Add linear regression lines
                se=TRUE)    # Add shaded confidence region

g1 <- set_panel_size(plot1)
ggsave(g1, file="/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/7_microbiomes_w_coral_size_lat/output/figures/raw_output/NEW_size_v_pd_color.svg")

Saving 7 x 7 in image


In [24]:
# By compartment: size vs. observed_otus
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Tissue")->data_corals_only_tissue
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Mucus")->data_corals_only_mucus
subset(data_scleractinia_compartments, BiologicalMatter == "Coral_Skeleton")->data_corals_only_skeleton

print("Tissue")
cor.test(as.numeric(as.character(data_corals_only_tissue$prop_Colony_maximum_diameter_universal)), as.numeric(as.character(data_corals_only_tissue$observed_otus)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)
print("Mucus")
cor.test(as.numeric(as.character(data_corals_only_mucus$prop_Colony_maximum_diameter_universal)), as.numeric(as.character(data_corals_only_mucus$observed_otus)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)
print("Skeleton")
cor.test(as.numeric(as.character(data_corals_only_skeleton$prop_Colony_maximum_diameter_universal)), as.numeric(as.character(data_corals_only_skeleton$observed_otus)),
         alternative = "two.sided", method = "pearson", conf.level = 0.95)

[1] "Tissue"



	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_tissue$prop_Colony_maximum_diameter_universal)) and as.numeric(as.character(data_corals_only_tissue$observed_otus))
t = -2.834, df = 197, p-value = 0.005075
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.32798270 -0.06049729
sample estimates:
       cor 
-0.1979218 


[1] "Mucus"



	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_mucus$prop_Colony_maximum_diameter_universal)) and as.numeric(as.character(data_corals_only_mucus$observed_otus))
t = -0.17186, df = 205, p-value = 0.8637
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.1481299  0.1245716
sample estimates:
       cor 
-0.0120023 


[1] "Skeleton"



	Pearson's product-moment correlation

data:  as.numeric(as.character(data_corals_only_skeleton$prop_Colony_maximum_diameter_universal)) and as.numeric(as.character(data_corals_only_skeleton$observed_otus))
t = -2.2025, df = 206, p-value = 0.02874
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.28190585 -0.01597025
sample estimates:
       cor 
-0.1516818 


In [26]:
# size vs. pd_whole_tree
#Clear plot
plot1<-0

#Plot
plot1 <-ggplot(data_scleractinia_compartments, aes(x=as.numeric(as.character(prop_Colony_maximum_diameter_universal)), y=as.numeric(as.character(observed_otus)), color=BiologicalMatter)) +
    geom_point(shape=1) +
    geom_smooth(method=lm,   # Add linear regression lines
                se=TRUE)    # Add shaded confidence region

g1 <- set_panel_size(plot1)
ggsave(g1, file="/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/7_microbiomes_w_coral_size_lat/output/figures/raw_output/NEW_size_v_observed_otus_color.svg")

Saving 7 x 7 in image
